In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import threading
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
np.set_printoptions(precision=3, suppress=True)

In [ ]:
df = pd.read_csv('/content/drive/My Drive/first.csv')
df2 = pd.read_csv('/content/drive/My Drive/Second.csv')
df3 = pd.read_csv('/content/drive/My Drive/final.csv')

In [ ]:
df.count()

CEP_x     11011
CEP_y     11011
CEP_z     11011
CEP_w     11011
CEV_x     11011
CEV_y     11011
CEV_z     11011
CEV_w     11011
CEA_x     11011
CEA_y     11011
CEA_z     11011
CEA_w     11011
CER_x     11011
CER_y     11011
CER_z     11011
CER_w     11011
CEAV_x    11011
CEAV_y    11011
CEAV_z    11011
CEAV_w    11011
LeftC     11011
RightC    11011
dtype: int64

In [ ]:
df2.count()

CEP_x      7520
CEP_y      7520
CEP_z      7520
CEAV_x     7520
CEAV_y     7520
CEAV_z     7520
Left_c     7520
Right_c    7520
dtype: int64

In [ ]:
df3.count()

CEP_x      14239
CEP_y      14239
CEP_z      14239
CEAV_x     14239
CEAV_y     14239
CEAV_z     14239
Left_c     14239
Right_c    14239
dtype: int64

In [ ]:
df3.columns

Index(['CEP_x', 'CEP_y', 'CEP_z', 'CEAV_x', 'CEAV_y', 'CEAV_z', 'Left_c',
       'Right_c'],
      dtype='object')

In [ ]:
df = df[['CEP_x', 'CEP_y', 'CEP_z', 'CEAV_x', 'CEAV_y', 'CEAV_z', 'LeftC', 'RightC']]

In [ ]:
df.rename(columns={'LeftC':'Left_c','RightC':'Right_c'},inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11011 entries, 0 to 11010
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CEP_x    11011 non-null  float64
 1   CEP_y    11011 non-null  float64
 2   CEP_z    11011 non-null  float64
 3   CEAV_x   11011 non-null  float64
 4   CEAV_y   11011 non-null  float64
 5   CEAV_z   11011 non-null  float64
 6   Left_c   11011 non-null  int64  
 7   Right_c  11011 non-null  int64  
dtypes: float64(6), int64(2)
memory usage: 688.3 KB


In [ ]:
c_df = pd.concat([df, df2,df3], axis=0)

In [ ]:
c_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32770 entries, 0 to 14238
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CEP_x    32770 non-null  float64
 1   CEP_y    32770 non-null  float64
 2   CEP_z    32770 non-null  float64
 3   CEAV_x   32770 non-null  float64
 4   CEAV_y   32770 non-null  float64
 5   CEAV_z   32770 non-null  float64
 6   Left_c   32770 non-null  int64  
 7   Right_c  32770 non-null  int64  
dtypes: float64(6), int64(2)
memory usage: 2.3 MB


In [ ]:
c_df

,CEP_x,CEP_y,CEP_z,CEAV_x,CEAV_y,CEAV_z,Left_c,Right_c
0,-0.02,0.89,-0.01,0.26,0.02,0.00,0,0
1,-0.02,0.89,-0.01,0.21,0.01,0.00,0,0
2,-0.02,0.89,-0.01,0.17,0.00,0.00,0,0
3,-0.02,0.89,-0.01,0.13,0.00,-0.01,0,0
4,-0.02,0.89,-0.01,0.10,0.00,-0.02,1,0
...,...,...,...,...,...,...,...,...
14234,0.02,0.88,-0.25,-0.98,0.10,-0.14,0,0
14235,0.02,0.88,-0.24,-1.04,0.12,-0.12,0,0
14236,0.01,0.87,-0.23,-1.08,0.14,-0.11,0,0
14237,0.01,0.87,-0.23,-1.11,0.14,-0.11,0,0


In [ ]:
c_df['move'] = c_df.apply(lambda row: 1 if row['Left_c'] == 1 and row['Right_c'] == 1 else 0, axis=1)


In [ ]:
c_df.drop(['Left_c', 'Right_c'], axis=1, inplace=True)

In [ ]:
#c_df.drop(['CEP_w', 'CEV_w','CEA_w','CEAV_w'], axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X = c_df.drop(['move'], axis=1)  # Features (all columns except 'move')
y = c_df['move']  # Target column

In [ ]:
#selected_features = ['CEP_x','CEP_y','CEP_z','CEV_x','CEV_z','CEA_z','CER_y','CER_w','CEAV_x']
#X_selected = df[selected_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# x is numpy array of shape (n,sample columns)
# y is numpy array of shape (n,...)
def roll_data(x,y,window_len,step=1):
    out_len=x.shape[0]-(window_len-1)
    retval=np.zeros((out_len,window_len,x.shape[1]))
    for c in range(window_len):
        retval[:out_len,c,:]=x[c:c+out_len,:]
    return retval,y[window_len-1:]

In [ ]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
print(X_train,y_train)

       CEP_x  CEP_y  CEP_z  CEAV_x  CEAV_y  CEAV_z
8476   -0.05   0.79   0.50    0.03   -0.12    0.01
4463   -0.01   0.90  -0.08    0.02    0.03   -0.02
2332   -0.05   0.82   0.35   -0.32    0.00    0.00
11816   0.02   0.91  -0.38    0.06    0.07   -0.04
374    -0.09   0.79   0.37   -0.18    0.00   -0.07
...      ...    ...    ...     ...     ...     ...
3044    0.02   0.90  -0.34   -0.08    0.00    0.07
11271  -0.05   0.79   0.12    0.18    0.15   -0.01
5390   -0.02   0.90  -0.20    0.07   -0.02   -0.07
860    -0.05   0.81   0.58    0.16   -0.14   -0.07
4784   -0.03   0.87  -0.28   -0.26    0.02    0.10

[26216 rows x 6 columns] 8476     0
4463     0
2332     0
11816    1
374      0
        ..
3044     1
11271    0
5390     1
860      0
4784     0
Name: move, Length: 26216, dtype: int64


In [ ]:
X_train=X_train.to_numpy()
y_train=y_train.to_numpy()

print("Final dataset:",X_train.shape,y_train.shape)

Final dataset: (26216, 6) (26216,)


In [ ]:
unroll_amount=512

X_train,y_train=roll_data(X_train,y_train,unroll_amount)

X_train_unshuffled=X_train
y_train_unshuffled=y_train
X_train,y_train=unison_shuffled_copies(X_train,y_train)

In [ ]:
model=tf.keras.Sequential([
    layers.Input(shape=(X_train.shape[1:])),
    layers.BatchNormalization(axis=-1),
    layers.Flatten(),
    layers.Dense(256,activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(64,activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(1,activation='sigmoid') # rectified unit so as not to return any rpm <0
]
)

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_1 (Batc  (None, 512, 6)           24        
 hNormalization)                                                 
                                                                 
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_3 (Dense)             (None, 256)               786688    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                16448     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                      

In [ ]:
def train_fn():
    model.fit(X_train,y_train,epochs=20,batch_size=512,shuffle=True)

In [ ]:
X_test=X_test.to_numpy()
y_test=y_test.to_numpy()

print("Final dataset:",X_test.shape,y_test.shape)



X_test,y_test=roll_data(X_test,y_test,unroll_amount)

X_test_unshuffled=X_test
y_test_unshuffled=y_test
X_test,y_test=unison_shuffled_copies(X_test,y_test)

Final dataset: (6554, 6) (6554,)


In [ ]:
train_thread=threading.Thread(target=train_fn)
train_thread.start()
plt.show()
train_thread.join()  # Wait for the training to complete

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=512)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")
print(f"Test loss: {test_loss}")

Epoch 1/20
51/51 [==============================] - 6s 98ms/step - loss: 0.6026 - accuracy: 0.6857
Epoch 2/20
51/51 [==============================] - 7s 134ms/step - loss: 0.2720 - accuracy: 0.8927
Epoch 3/20
51/51 [==============================] - 5s 95ms/step - loss: 0.1237 - accuracy: 0.9571
Epoch 4/20
51/51 [==============================] - 6s 111ms/step - loss: 0.0587 - accuracy: 0.9827
Epoch 5/20
51/51 [==============================] - 6s 121ms/step - loss: 0.0385 - accuracy: 0.9874
Epoch 6/20
51/51 [==============================] - 5s 93ms/step - loss: 0.0259 - accuracy: 0.9917
Epoch 7/20
51/51 [==============================] - 6s 125ms/step - loss: 0.0183 - accuracy: 0.9944
Epoch 8/20
51/51 [==============================] - 5s 106ms/step - loss: 0.0199 - accuracy: 0.9936
Epoch 9/20
51/51 [==============================] - 5s 99ms/step - loss: 0.0137 - accuracy: 0.9961
Epoch 10/20
51/51 [==============================] - 8s 164ms/step - loss: 0.0122 - accuracy: 0.9961
Epo

from google.colab import files

In [ ]:
#model.save('outputs/saved_model_{epoch}')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('model_fin.tflite', 'wb') as f:
    f.write(tflite_model)
           # with open(f'outputs/model_please.tflite', 'wb') as f:
            #    f.write(tflite_model)

31/51 [=================>............] - ETA: 1s - loss: 0.0266 - accuracy: 0.9907

Exception in thread Thread-11 (train_fn):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-43-72e4501e3900>", line 2, in train_fn
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1685, in fit
    tmp_logs = self.train_function(iterator)
TypeError: 'NoneType' object is not callable


In [ ]:
from google.colab import files
files.download('model_fin.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install tf2onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 13.4 MB/s eta 0:00:00


In [ ]:
import tf2onnx

# Converting the model to ONNX format
onnx_model, _ = tf2onnx.convert.from_keras(model,
                                           input_signature=[tf.TensorSpec(shape=[None,X_train.shape[1],6], dtype=tf.float32, name="input")],
                                           opset=13)

# Saving the ONNX model to a file
with open("model_finon.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
files.download('model_finon.onnx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>